In [ ]:
from datasets.gait_fc_dataset import GaitFCDatasetV2
from models.gait_fc import GaitFCV2
import torch
from losses.arcloss import ArcFace, DistCrossEntropy
from torch.nn import functional as F


In [ ]:
train_dataset = GaitFCDatasetV2('gait_fc_dataset/train')
print(len(train_dataset))

In [ ]:
batch_size = 32
epochs = 50
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
device = torch.device('cpu')
model = GaitFCV2()
model.to(device)
model.train()

arc = ArcFace(256, 400, s=32, m=0.5)


In [ ]:
margin = 1.
criteron = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)

In [ ]:
start_epoch = 0
print('start training !')
for epoch in range(start_epoch, epochs):
    model.train()
    losses = []
    iter = 0
    for batch_idx, (a_v, a_gei, target) in enumerate(train_loader):
        a_v = a_v.to(device).float()
        a_gei = a_gei.to(device).float()
        optimizer.zero_grad()
        anchor_out = model(a_v, a_gei)
        print(target)
        anchor_out = arc(anchor_out, target)
        loss = criteron(anchor_out, target)
        loss.backward()
        optimizer.step()
        print(f'Epoch: [%d]/[%d] Training, %.2f%%,{iter}/{len(train_dataset)}, Loss={loss.item()}' % (epoch, epochs, iter*100/len(train_dataset)))
        iter += len(a_v)
    scheduler.step()



In [ ]:
torch.save(model.state_dict(), 'src/gait_fcv2_epoch_50.pt')